In [1]:
from tinygrad.densetensor import DenseTensor
from tinygrad.sparsetensor import SparseTensor
import numpy as np

%load_ext autoreload
%autoreload 2

DEVICE:GPU


In [2]:
x_init = np.random.randn(2,6).astype(np.float32)
x2_init = np.random.randn(3).astype(np.float32)
U_init = np.random.randn(3,3).astype(np.float32)
V_init = np.random.randn(3,3).astype(np.float32)
W_init = np.random.randn(6,3).astype(np.float32)
m_init = np.random.randn(1,3).astype(np.float32)

In [3]:
x = DenseTensor(x_init)
W = DenseTensor(W_init)
m = DenseTensor(m_init)
out = x.dot(W).relu()
out = out.logsoftmax()
out = out.mul(m).add(m).sum()
out.backward()

out.cpu().data, x

(array([0.85907143], dtype=float32),
 <DenseTensor <GPUBuffer with shape (2, 6)> with grad <GPUBuffer with shape (2, 6)>>)

x2 = DenseTensor(x2_init)#.gpu()
W = SparseTensor(W_init)
out = W.dot(x2).relu().sum()

out.backward()

out.cpu().data, x

In [4]:
import numpy as np
import pyopencl as cl

mf = cl.mem_flags

In [5]:
dim1 = 2
dim2 = 4
dim3 = 3
bs = dim3

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx,
        properties=cl.command_queue_properties.PROFILING_ENABLE)

sparsity = 0.4

a = np.zeros((dim1,dim2))
b = np.zeros((dim2,dim3)).astype(np.float32)

a.shape, b.shape

((2, 4), (4, 3))

In [6]:
def fill_sparse(mat, sparsity=0.1):
    indices = np.array(range(mat.shape[1]))
    nrows = int(mat.shape[1]*sparsity)
    for row in range(mat.shape[0]):
        lim = nrows #+ int(np.random.random()*3)
        mat[row][np.random.permutation(indices)[:lim]] = np.random.random(lim)
    return mat

a = fill_sparse(a, sparsity).astype(np.float32)
b = fill_sparse(b, sparsity).astype(np.float32)

In [7]:
a

array([[0.        , 0.10336889, 0.        , 0.        ],
       [0.56775475, 0.        , 0.        , 0.        ]], dtype=float32)

In [8]:
b

array([[0.        , 0.        , 0.55664927],
       [0.14582244, 0.        , 0.        ],
       [0.        , 0.        , 0.6065191 ],
       [0.36334926, 0.        , 0.        ]], dtype=float32)

In [9]:
x2_init.T

array([-0.93136215,  0.30850452,  0.943738  ], dtype=float32)

In [10]:
mult = a.dot(b)
mult.shape

(2, 3)

In [11]:
mult.shape

(2, 3)

In [12]:
def to_data(mat):
    all_rows = []
    all_idxs = []
    all_nnzs = []
    for row in range(mat.shape[0]):
        rowdata = []
        colidxs = []
        all_nnzs.append(0)
        for col in range(mat.shape[1]):
            val = mat[row][col]
            if val != 0:
                rowdata.append(val)
                colidxs.append(col)
                all_nnzs[-1] += 1
        all_rows.append(rowdata)
        all_idxs.append(colidxs)
    
    ellwidth = min(int(np.sqrt(np.max(all_nnzs))+1)**2, mat.shape[1])
    #all_rows = np.array(all_rows)#.astype(np.float32).flatten()           
    for row in range(mat.shape[0]):
        #print(row, all_rows)
        all_rows[row] = np.array(all_rows[row])
        all_rows[row].resize(ellwidth)
        all_idxs[row] = np.array(all_idxs[row])
        all_idxs[row].resize(ellwidth)
        #print(all_idxs[row])
    all_rows = np.array(all_rows)
    all_idxs = np.array(all_idxs)
    all_nnzs = np.array(all_nnzs)
    
    while (not all_rows[:,-1].any()):
        all_rows = all_rows[:,:-1]
        all_idxs = all_idxs[:,:-1]
        ellwidth -= 1
        
    
    all_rows = np.array(all_rows).astype(np.float32).flatten()
    all_idxs = np.array(all_idxs).astype(np.uint32).flatten()
    
    all_nnzs = np.array(all_nnzs).astype(np.uint32)
    
    
    return all_rows, all_idxs, all_nnzs, ellwidth

In [13]:
def to_dense(data, cols, nnzs, ellw, shape):
    out = np.zeros(shape)
    for row in range(shape[0]):
        for icol in range(nnzs[row]):
            out[row,cols[row*ellw+icol]] = data[row*ellw+icol]
    return out

In [14]:
adata, acols, annz, ellwa = to_data(a)
adata, acols, annz, ellwa

(array([0.10336889, 0.56775475], dtype=float32),
 array([1, 0], dtype=uint32),
 array([1, 1], dtype=uint32),
 1)

In [15]:
adatat, acolst, annzt, ellwat = to_data(a.T)
adatat, acolst, annzt, ellwat

(array([0.56775475, 0.10336889, 0.        , 0.        ], dtype=float32),
 array([1, 0, 0, 0], dtype=uint32),
 array([1, 1, 0, 0], dtype=uint32),
 1)

In [16]:
bdata, bcols, bnnz, ellwb = to_data(b)
bdata, bcols, bnnz, ellwb

(array([0.55664927, 0.14582244, 0.6065191 , 0.36334926], dtype=float32),
 array([2, 0, 2, 0], dtype=uint32),
 array([1, 1, 1, 1], dtype=uint32),
 1)

In [17]:
bdatat, bcolst, bnnzt, ellwbt = to_data(b.T)
adatat, bcolst, bnnzt, ellwbt

(array([0.56775475, 0.10336889, 0.        , 0.        ], dtype=float32),
 array([1, 3, 0, 0, 0, 2], dtype=uint32),
 array([2, 0, 2], dtype=uint32),
 2)

In [18]:
adense = to_dense(adata, acols, annz, ellwa, a.shape)

In [19]:
adenset = to_dense(adatat, acolst, annzt, ellwat, a.T.shape)

In [20]:
bdense = to_dense(bdata, bcols, bnnz, ellwb, b.shape)

In [21]:
bdenset = to_dense(bdatat, bcolst, bnnzt, ellwbt, b.T.shape)

In [22]:
adense

array([[0.        , 0.10336889, 0.        , 0.        ],
       [0.56775475, 0.        , 0.        , 0.        ]])

In [23]:
adenset.T == adense

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [24]:
bdenset.T == bdense

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [25]:
a

array([[0.        , 0.10336889, 0.        , 0.        ],
       [0.56775475, 0.        , 0.        , 0.        ]], dtype=float32)

In [26]:
a == adense

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [27]:
a.shape

(2, 4)

In [28]:
adata.shape, acols.shape, annz.shape, ellwa

((2,), (2,), (2,), 1)

In [29]:
#acols = acols.astype(np.uint32)
#annz = annz.astype(np.uint32)

In [30]:
adata, acols, annz, b

(array([0.10336889, 0.56775475], dtype=float32),
 array([1, 0], dtype=uint32),
 array([1, 1], dtype=uint32),
 array([[0.        , 0.        , 0.55664927],
        [0.14582244, 0.        , 0.        ],
        [0.        , 0.        , 0.6065191 ],
        [0.36334926, 0.        , 0.        ]], dtype=float32))

## MatMul (Sparse-Dense)

adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // SPARSE x DENSE
    __kernel void matmul2(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint ncols,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);
      

      uint nnz    = rowNnz[gid];
      float sum = 0;
      for (uint gid2 = 0; gid2 < ncols; gid2++) {
        for (uint i = 0; i < nnz; i++) {
          uint index   = (gid * ellwidth) + i;
          uint col     = colIdx[index];
          float aval  = matData[index];
          float xval  = vector_x[col*ncols+gid2];
          //if (gid==0 && gid2==2)
          //  printf("aval, xval: %.2f,%.2f: (%i,%i) \\n", aval, xval, col, index);
          sum  += aval * xval;
        }
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
        vector_y[gid*ncols+gid2] = sum;
      }
    }""").build()

In [31]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // SPARSE x DENSE
    __kernel void matmul2(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint ncols,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);
      

      uint nnz    = rowNnz[gid];
      
      for (uint gid2 = 0; gid2 < ncols; gid2++) {
        float sum = 0;
        for (uint i = 0; i < nnz; i++) {
          uint index   = (gid * ellwidth) + i;
          uint col     = colIdx[index];
          float aval  = matData[index];
          uint xidx = col*ncols+gid2;
          float xval  = vector_x[xidx];
          if (gid==0 && gid2==1)
            printf("aval, xval: %.2f,%.2f: (%i,%i) - %i \\n", aval, xval, col, index, xidx);
          sum  += aval * xval;
        }
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
        vector_y[gid*ncols+gid2] = sum;
      }
    }""").build()

In [32]:
a.shape, b.shape

((2, 4), (4, 3))

In [33]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [34]:
rows = a.shape[0]

In [35]:
mult = mult.astype(np.float32)

In [36]:
outshape = (a.shape[0], b.shape[1])
outshape

(2, 3)

In [37]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod(outshape)*4)
knl = prg.matmul2  # Use this Kernel object for repeated calls
knl(queue, [outshape[0]], None, adata_buf, acols_buf, annzs_buf, np.uint32(ellwa), np.uint32(outshape[1]), b_buf, res_buf)

res_np = np.zeros(outshape).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)

aval, xval: 0.10,0.00: (1,0) - 4 


In [38]:
(res_np-mult).sum()

0.0

In [39]:
a

array([[0.        , 0.10336889, 0.        , 0.        ],
       [0.56775475, 0.        , 0.        , 0.        ]], dtype=float32)

In [40]:
b

array([[0.        , 0.        , 0.55664927],
       [0.14582244, 0.        , 0.        ],
       [0.        , 0.        , 0.6065191 ],
       [0.36334926, 0.        , 0.        ]], dtype=float32)

In [41]:
res_buf

In [42]:
res_np

array([[0.0150735 , 0.        , 0.        ],
       [0.        , 0.        , 0.31604028]], dtype=float32)

In [43]:
mult

array([[0.0150735 , 0.        , 0.        ],
       [0.        , 0.        , 0.31604028]], dtype=float32)

In [44]:
res_np==mult

array([[ True,  True,  True],
       [ True,  True,  True]])

In [45]:
res_np.shape

(2, 3)

In [46]:
mult.shape

(2, 3)

## MatMul (dense * sparse)

In [47]:
bdata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bdata)
bcols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bcols)
bnnzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bnnz)
bdatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bdatat)
bcolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bcolst)
bnnzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=bnnzt)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // DENSE x SPARSE
    __kernel void matmul(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            uint   ellwidth,
                            uint   mwidth,
                            uint   ncols,
                            __global  float* vector_x,    // INPUT
                            __global  float* vector_y    // OUTPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);

      for (uint gid2 = 0; gid2 < ncols; gid2++) {
        uint nnz = rowNnz[gid2];
        float sum = 0;
        for (uint i = 0; i < nnz; i++) {
          uint index   = (gid2 * ellwidth) + i;
          uint col     = colIdx[index];
          float aval  = matData[index];
          float xval  = vector_x[gid*mwidth+col];
          sum  += aval * xval;
          if (gid==0 && gid2==0)
            printf("aval, xval: %.2f,%.2f - %.2f: (%i,%i) \\n", aval, xval, sum, col, index);
        }
        //printf("SUM/NNZ: %.2f %i \\n", sum, nnz);
        vector_y[gid*ncols+gid2] = sum;
      }
    }""").build()

In [48]:
a.shape, b.shape

((2, 4), (4, 3))

In [49]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [50]:
rows = a.shape[0]

In [51]:
mult = mult.astype(np.float32)

In [52]:
outshape = np.array([a.shape[0], b.shape[1]])
outshape

array([2, 3])

In [53]:
b.T

array([[0.        , 0.14582244, 0.        , 0.36334926],
       [0.        , 0.        , 0.        , 0.        ],
       [0.55664927, 0.        , 0.6065191 , 0.        ]], dtype=float32)

In [54]:
a.T

array([[0.        , 0.56775475],
       [0.10336889, 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ]], dtype=float32)

In [55]:
outshape.T

array([2, 3])

In [56]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod(outshape)*4)
knl = prg.matmul  # Use this Kernel object for repeated calls
knl(queue, [outshape.T[0]], None, bdatat_buf, bcolst_buf, bnnzst_buf, np.uint32(ellwbt), np.uint32(b.shape[0]), np.uint32(outshape.T[1]), a_buf, res_buf)

res_np = np.zeros(outshape).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)

aval, xval: 0.15,0.10 - 0.02: (1,0) 
aval, xval: 0.36,0.00 - 0.02: (3,1) 


In [57]:
(res_np-mult).sum()

0.0

In [58]:
a

array([[0.        , 0.10336889, 0.        , 0.        ],
       [0.56775475, 0.        , 0.        , 0.        ]], dtype=float32)

In [59]:
b

array([[0.        , 0.        , 0.55664927],
       [0.14582244, 0.        , 0.        ],
       [0.        , 0.        , 0.6065191 ],
       [0.36334926, 0.        , 0.        ]], dtype=float32)

In [60]:
res_buf

In [61]:
res_np

array([[0.0150735 , 0.        , 0.        ],
       [0.        , 0.        , 0.31604028]], dtype=float32)

In [62]:
mult

array([[0.0150735 , 0.        , 0.        ],
       [0.        , 0.        , 0.31604028]], dtype=float32)

In [63]:
res_np==mult

array([[ True,  True,  True],
       [ True,  True,  True]])

In [64]:
res_np-mult

array([[0., 0., 0.],
       [0., 0., 0.]], dtype=float32)

In [65]:
res_np.shape

(2, 3)

In [66]:
mult.shape

(2, 3)

# Matmult Dense Dense

In [67]:
b_buf2 = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // multilplies x by y WITH Y TRANSPOSED INDEXING
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize
                          ) { // LOCAL SHARED BUFFER
      uint isize = get_global_size(0);
      uint osize = get_global_size(1);
      int gidx = get_global_id(0); // row
      int gidy = get_global_id(1); // col

      float ret = 0.0;
      for (int i = 0; i < msize; i++) {
        uint xidx = gidx*msize+i; 
        float xval = x[xidx];
        uint yidx = osize*i+gidy;
        float yval = y[yidx];
        ret += xval*yval;
        if (gidx==0 && gidy==0)
          printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, res, xidx, yidx);
      }

      //if (gidx==0&&gidy==0)
      //  printf("\\nsum:%.2f", ret);
      res[gidx * osize + gidy] = ret;
    }""").build()

In [68]:
a.shape, b.shape

((2, 4), (4, 3))

In [69]:
rows = a.shape[0]

In [70]:
mult = mult.astype(np.float32)

In [71]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [rows,b.shape[1]], None, a_buf, b_buf2, res_buf, np.uint32(a.shape[1]))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)


mult: 0.00 x 0.00 - 0.00  -- 0/0
mult: 0.10 x 0.15 - 0.00  -- 1/3
mult: 0.00 x 0.00 - 0.00  -- 2/6
mult: 0.00 x 0.36 - 0.00  -- 3/9

In [72]:
(res_np-mult).sum()

0.0

In [73]:
a

array([[0.        , 0.10336889, 0.        , 0.        ],
       [0.56775475, 0.        , 0.        , 0.        ]], dtype=float32)

In [74]:
b

array([[0.        , 0.        , 0.55664927],
       [0.14582244, 0.        , 0.        ],
       [0.        , 0.        , 0.6065191 ],
       [0.36334926, 0.        , 0.        ]], dtype=float32)

In [75]:
res_np

array([[0.0150735 , 0.        , 0.        ],
       [0.        , 0.        , 0.31604028]], dtype=float32)

In [76]:
a.dot(b)

array([[0.0150735 , 0.        , 0.        ],
       [0.        , 0.        , 0.31604028]], dtype=float32)

In [77]:
res_np==mult

array([[ True,  True,  True],
       [ True,  True,  True]])

In [78]:
res_np.shape

(2, 3)

In [79]:
mult.shape

(2, 3)

# Matmult Dense Transposed

In [80]:
b

array([[0.        , 0.        , 0.55664927],
       [0.14582244, 0.        , 0.        ],
       [0.        , 0.        , 0.6065191 ],
       [0.36334926, 0.        , 0.        ]], dtype=float32)

In [81]:
c=np.zeros(b.T.shape)
bt = b.T
for row in range(bt.shape[0]):
    for col in range(bt.shape[1]):
        c[row][col] = bt[row][col]

In [82]:
bt

array([[0.        , 0.14582244, 0.        , 0.36334926],
       [0.        , 0.        , 0.        , 0.        ],
       [0.55664927, 0.        , 0.6065191 , 0.        ]], dtype=float32)

In [83]:
c

array([[0.        , 0.14582244, 0.        , 0.36334926],
       [0.        , 0.        , 0.        , 0.        ],
       [0.55664927, 0.        , 0.6065191 , 0.        ]])

In [84]:
b_buf2 = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // multilplies x by y WITH Y TRANSPOSED INDEXING
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize
                          ) { // LOCAL SHARED BUFFER
      uint isize = get_global_size(0);
      uint osize = get_global_size(1);
      int gidx = get_global_id(0); // row
      int gidy = get_global_id(1); // col

      float ret = 0.0;
      for (int i = 0; i < msize; i++) {
        uint xidx = gidx*msize+i;
        float xval = x[xidx];
        uint yidx = msize*gidy+i;
        float yval = y[yidx];
        ret += xval*yval;
        if (gidx==0 && gidy==0)
          printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, res, xidx, yidx);
      }

      //if (gidx==0&&gidy==0)
      //  printf("\\nsum:%.2f", ret);
      res[gidx * osize + gidy] = ret;
    }""").build()

In [85]:
a.shape, b.T.shape

((2, 4), (3, 4))

In [86]:
rows = a.shape[0]

In [87]:
mult = mult.astype(np.float32)

In [88]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [rows,b.shape[1]], None, a_buf, b_buf2, res_buf, np.uint32(a.shape[1]))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)


mult: 0.00 x 0.00 - 0.00  -- 0/0
mult: 0.10 x 0.15 - 0.00  -- 1/1
mult: 0.00 x 0.00 - 0.00  -- 2/2
mult: 0.00 x 0.36 - 0.00  -- 3/3

In [89]:
(res_np-mult).sum()

0.0

In [90]:
a

array([[0.        , 0.10336889, 0.        , 0.        ],
       [0.56775475, 0.        , 0.        , 0.        ]], dtype=float32)

In [91]:
b

array([[0.        , 0.        , 0.55664927],
       [0.14582244, 0.        , 0.        ],
       [0.        , 0.        , 0.6065191 ],
       [0.36334926, 0.        , 0.        ]], dtype=float32)

In [92]:
res_np

array([[0.0150735 , 0.        , 0.        ],
       [0.        , 0.        , 0.31604028]], dtype=float32)

In [93]:
a.dot(b)

array([[0.0150735 , 0.        , 0.        ],
       [0.        , 0.        , 0.31604028]], dtype=float32)

In [94]:
res_np==mult

array([[ True,  True,  True],
       [ True,  True,  True]])

In [95]:
res_np.shape

(2, 3)

In [96]:
mult.shape

(2, 3)

# Matmult Transposed Dense

In [97]:
b

array([[0.        , 0.        , 0.55664927],
       [0.14582244, 0.        , 0.        ],
       [0.        , 0.        , 0.6065191 ],
       [0.36334926, 0.        , 0.        ]], dtype=float32)

In [98]:
c=np.zeros(a.T.shape)
at = a.T
for row in range(at.shape[0]):
    for col in range(at.shape[1]):
        c[row][col] = at[row][col]

In [99]:
at

array([[0.        , 0.56775475],
       [0.10336889, 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ]], dtype=float32)

In [100]:
c

array([[0.        , 0.56775475],
       [0.10336889, 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ]])

In [101]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)

prg = cl.Program(ctx, """
    // multilplies x TRANSPOSED by y (dense-dense)
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize,
                          uint isize
                          ) { // LOCAL SHARED BUFFER
      uint osize = get_global_size(0);
      int gidy = get_global_id(0); // row
      
      for (uint gidx = 0; gidx < isize; gidx++) {
        float ret = 0.0;
        for (uint i = 0; i < msize; i++) {
          uint xidx = i*isize+gidx;
          float xval = x[xidx];
          uint yidx = osize*i+gidy;
          float yval = y[yidx];
          ret += xval*yval;
          if (gidx==0 && gidy==0)
            printf("\\nmult: %.2f x %.2f - %.2f  -- %i/%i", xval, yval, ret, xidx, yidx);
        }
        //if (gidx==0&&gidy==0)
        //  printf("\\nsum:%.2f", ret);
        res[gidx * osize + gidy] = ret;
      }
    }""").build()

In [102]:
a.shape, b.T.shape

((2, 4), (3, 4))

In [103]:
rows = a.shape[0]

In [104]:
mult = mult.astype(np.float32)

In [105]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [b.shape[1]], None, a_buf, b_buf, res_buf, np.uint32(a.shape[1]), np.uint32(rows))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)


mult: 0.00 x 0.00 - 0.00  -- 0/0
mult: 0.10 x 0.15 - 0.02  -- 2/3
mult: 0.00 x 0.00 - 0.02  -- 4/6
mult: 0.00 x 0.36 - 0.02  -- 6/9

In [106]:
(res_np-mult).sum()

0.0

In [107]:
a

array([[0.        , 0.10336889, 0.        , 0.        ],
       [0.56775475, 0.        , 0.        , 0.        ]], dtype=float32)

In [108]:
b

array([[0.        , 0.        , 0.55664927],
       [0.14582244, 0.        , 0.        ],
       [0.        , 0.        , 0.6065191 ],
       [0.36334926, 0.        , 0.        ]], dtype=float32)

In [109]:
res_np

array([[0.0150735 , 0.        , 0.        ],
       [0.        , 0.        , 0.31604028]], dtype=float32)

In [110]:
a.dot(b)

array([[0.0150735 , 0.        , 0.        ],
       [0.        , 0.        , 0.31604028]], dtype=float32)

In [111]:
res_np==mult

array([[ True,  True,  True],
       [ True,  True,  True]])

In [112]:
res_np.shape

(2, 3)

In [113]:
mult.shape

(2, 3)

# Matmult Dense Transposed2

In [114]:
b_buf2 = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)

prg = cl.Program(ctx, """
    // multilplies x by y WITH Y TRANSPOSED INDEXING
    __kernel void matmul0(__global  float* x,     // INPUT MATRIX DATA
                          __global  float* y,    // INPUT
                          __global  float* res,    // INPUT
                          uint msize,
                          uint osize
                          ) { // LOCAL SHARED BUFFER
      uint isize = get_global_size(0);
      // osize = get_global_size(1);
      int gidx = get_global_id(0); // col
      // int gidy = get_global_id(1); // row

      for (uint gidy = 0; gidy < osize; gidy++) {
        float ret = 0.0;
        for (uint i = 0; i < msize; i++) {
          ret += x[gidx*msize+i]*y[i*osize+gidy];
          if (gidx==0 && gidy==0)
            printf("\\nmult: %.2f x %.2f - %.2f", x[gidx*msize+i],y[i*msize+gidy], ret);
        }

        //if (gidx==0&&gidy==0)
        //  printf("\\nsum:%.2f", ret);
        res[gidx * osize + gidy] = ret;
      }
    }""").build()

In [115]:
a.shape, b.shape

((2, 4), (4, 3))

In [116]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [117]:
rows = a.shape[0]

In [118]:
mult = mult.astype(np.float32)

In [119]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.matmul0  # Use this Kernel object for repeated calls
knl(queue, [rows], None, a_buf, b_buf2, res_buf, np.uint32(a.shape[1]), np.uint32(b.shape[1]))

res_np = np.zeros([rows,b.shape[1]]).astype(np.float32)
cl.enqueue_copy(queue, res_np, res_buf)


mult: 0.00 x 0.00 - 0.00
mult: 0.10 x 0.00 - 0.02
mult: 0.00 x 0.61 - 0.02
mult: 0.00 x 0.00 - 0.02

In [120]:
(res_np-mult).sum()

0.0

In [121]:
a

array([[0.        , 0.10336889, 0.        , 0.        ],
       [0.56775475, 0.        , 0.        , 0.        ]], dtype=float32)

In [122]:
b.T

array([[0.        , 0.14582244, 0.        , 0.36334926],
       [0.        , 0.        , 0.        , 0.        ],
       [0.55664927, 0.        , 0.6065191 , 0.        ]], dtype=float32)

In [123]:
a[0]

array([0.        , 0.10336889, 0.        , 0.        ], dtype=float32)

In [124]:
b.T[0]

array([0.        , 0.14582244, 0.        , 0.36334926], dtype=float32)

In [125]:
res_buf

In [126]:
res_np

array([[0.0150735 , 0.        , 0.        ],
       [0.        , 0.        , 0.31604028]], dtype=float32)

In [127]:
a.dot(b)

array([[0.0150735 , 0.        , 0.        ],
       [0.        , 0.        , 0.31604028]], dtype=float32)

In [128]:
res_np==mult

array([[ True,  True,  True],
       [ True,  True,  True]])

In [129]:
res_np.shape

(2, 3)

In [130]:
mult.shape

(2, 3)

## Weight update kernel

In [131]:
bs = 4

In [132]:
a

array([[0.        , 0.10336889, 0.        , 0.        ],
       [0.56775475, 0.        , 0.        , 0.        ]], dtype=float32)

In [133]:
dim = 8
topk = 2

x = np.random.rand(bs,dim).astype(np.float32)
y = np.random.rand(bs,dim).astype(np.float32)
x.shape,y.shape, topk

((4, 8), (4, 8), 2)

x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                             __global  float* y,    // INPUT
                             __global  float* xout,    // INPUT
                             uint topk,
                             __global  uint* xoutidx,    // INPUT
                             __global  uint* youtidx    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);
      uint bs = get_global_size(1);
      uint gid2 = get_global_id(1);

      uint idx = n*gid2+gid;

      float valx = x[idx];
      float valy = y[idx];
      uint posx = 0;
      uint posy = 0;
      for (uint i = 0; i < n; i++) {
        uint idx2 = n*gid2+i;
        float tempval = x[idx2];
        float tempval2 = y[idx2];
        bool larger = tempval > valx;
        bool larger2 = tempval2 > valy;

        barrier(CLK_GLOBAL_MEM_FENCE);
        posx += (larger)?1:0;
        posy += (larger2)?1:0;
        barrier(CLK_GLOBAL_MEM_FENCE);
      }
      barrier(CLK_GLOBAL_MEM_FENCE);
      //printf("posx:%i", posx);
      if (posx < topk) {
        xoutidx[posx+topk*gid2] = gid;
      }
      if (posy < topk) {
        youtidx[posy+topk*gid2] = gid;
      }
      barrier(CLK_GLOBAL_MEM_FENCE);
      if (gid < topk) {
        for (uint j=0; j<topk; j++) {
          float res = x[xoutidx[gid+topk*gid2]+gid2*n] * y[youtidx[j+topk*gid2]+gid2*n];
          //printf("\\nJ:%i  gid:%i", j, gid);
          //printf("\\nRES:%.2f - %i - %i -  %.2f - %.2f",res, xoutidx[gid+topk*gid2], youtidx[j+topk*gid2], x[xoutidx[gid+topk*gid2]+gid2*n], y[youtidx[j+topk*gid2]+gid2*n]);
          barrier(CLK_GLOBAL_MEM_FENCE);
          xout[gid2*topk*topk+j*topk+gid] = res;
          barrier(CLK_GLOBAL_MEM_FENCE);
          
        }
      }
      barrier(CLK_GLOBAL_MEM_FENCE);
    }""").build()

In [134]:
x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
#x_cp_buft = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*topk*4)
#x_idx_buft = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)
#y_idx_buft = cl.Buffer(ctx, mf.WRITE_ONLY, bs*topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                             __global  float* y,    // INPUT
                             __global  float* xout,    // INPUT
                             uint topk,
                             uint bs,
                             __global  uint* xoutidx,    // INPUT
                             __global  uint* youtidx    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);
      //uint bs = get_global_size(1);
      //uint gid2 = get_global_id(1);

      for (uint gid2=0; gid2<bs; gid2++){
        uint idx = n*gid2+gid;

        float valx = x[idx];
        float valy = y[idx];
        uint posx = 0;
        uint posy = 0;
        for (uint i = 0; i < n; i++) {
          uint idx2 = n*gid2+i;
          float tempval = x[idx2];
          float tempval2 = y[idx2];
          bool larger = tempval > valx;
          bool larger2 = tempval2 > valy;

          posx += (larger)?1:0;
          posy += (larger2)?1:0;
        }
        //printf("posx:%i", posx);
        if (posx < topk) {
        xoutidx[posx+topk*gid2] = gid;
        }
        if (posy < topk) {
          youtidx[posy+topk*gid2] = gid;
        }
      }
      for (uint gid2=0; gid2<bs; gid2++){
        if (gid < topk) {
          for (uint j=0; j<topk; j++) {
            float res = x[xoutidx[gid+topk*gid2]+gid2*n] * y[youtidx[j+topk*gid2]+gid2*n];
            //printf("\\nJ:%i  gid:%i", j, gid);
            //printf("\\nRES:%.2f - %i - %i -  %.2f - %.2f",res, xoutidx[gid+topk*gid2], youtidx[j+topk*gid2], x[xoutidx[gid+topk*gid2]+gid2*n], y[youtidx[j+topk*gid2]+gid2*n]);
            //barrier(CLK_GLOBAL_MEM_FENCE);
            xout[gid2*topk*topk+j*topk+gid] = res;
          }
        }
      }
    }""").build()

In [135]:
knl = prg.genwupdate2  # Use this Kernel object for repeated calls
evt = knl(queue, [dim], None, x_buf, y_buf, x_cp_buf, np.uint32(topk), np.uint32(bs), x_idx_buf, y_idx_buf)

#evt.wait()
resx = np.zeros(bs*topk*topk).astype(np.float32)
resxidx = np.zeros(bs*topk).astype(np.uint32)
resyidx = np.zeros(bs*topk).astype(np.uint32)

cl.enqueue_copy(queue, resx, x_cp_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)

knl(queue, [dim], None, y_buf, x_buf, x_cp_buft, np.uint32(topk), np.uint32(bs), x_idx_buft, y_idx_buft)

#evt.wait()
resx = np.zeros(bs*topk*topk).astype(np.float32)
resxidx = np.zeros(bs*topk).astype(np.uint32)
resyidx = np.zeros(bs*topk).astype(np.uint32)

cl.enqueue_copy(queue, resx, x_cp_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)

In [136]:
x

array([[0.8130812 , 0.78479683, 0.58256865, 0.59758997, 0.6558741 ,
        0.5255338 , 0.28567916, 0.99740034],
       [0.14986885, 0.16550867, 0.26537052, 0.45639244, 0.5287076 ,
        0.80720717, 0.63659316, 0.75992817],
       [0.70254064, 0.5446952 , 0.02506934, 0.29217   , 0.10948461,
        0.7324262 , 0.95252955, 0.18555576],
       [0.6357195 , 0.7182958 , 0.9181955 , 0.4838693 , 0.9582339 ,
        0.8502439 , 0.49310943, 0.7299614 ]], dtype=float32)

In [137]:
y

array([[2.5641942e-01, 1.1095346e-01, 2.7817920e-01, 3.9182594e-01,
        6.3468790e-01, 9.4803840e-02, 5.9704864e-01, 6.3615394e-01],
       [7.0968986e-01, 3.0352470e-01, 9.5766854e-01, 8.8771707e-01,
        3.6237606e-01, 5.1863831e-02, 7.7793008e-01, 6.1140084e-01],
       [3.1428263e-02, 1.3467787e-01, 2.0725454e-01, 3.2820072e-02,
        5.9834760e-01, 6.0574991e-01, 9.6120524e-01, 5.3361040e-01],
       [3.4532932e-04, 1.4912307e-01, 2.3269188e-01, 3.9037102e-01,
        2.6819867e-01, 7.5664109e-01, 2.0246562e-01, 2.3121756e-01]],
      dtype=float32)

In [138]:
x.shape, y.shape

((4, 8), (4, 8))

In [139]:
resx

array([0.63450015, 0.5172448 , 0.6330379 , 0.5160528 , 0.7730369 ,
       0.7277593 , 0.71657157, 0.6746012 , 0.9155764 , 0.7040119 ,
       0.57699466, 0.4436671 , 0.7250391 , 0.6947444 , 0.37406674,
       0.3584369 ], dtype=float32)

In [140]:
resx.reshape(bs,topk,topk)

array([[[0.63450015, 0.5172448 ],
        [0.6330379 , 0.5160528 ]],

       [[0.7730369 , 0.7277593 ],
        [0.71657157, 0.6746012 ]],

       [[0.9155764 , 0.7040119 ],
        [0.57699466, 0.4436671 ]],

       [[0.7250391 , 0.6947444 ],
        [0.37406674, 0.3584369 ]]], dtype=float32)

In [141]:
resxidx

array([7, 0, 5, 7, 6, 5, 4, 2], dtype=uint32)

In [142]:
resyidx

array([7, 4, 2, 3, 6, 5, 5, 3], dtype=uint32)

In [143]:
idx = 1
xy0 = x[idx].reshape(dim,1)*y[idx]
xy0.shape

(8, 8)

In [144]:
xy0[3][7]

0.27903873

## Weight update kernel new

In [145]:
b

array([[0.        , 0.        , 0.55664927],
       [0.14582244, 0.        , 0.        ],
       [0.        , 0.        , 0.6065191 ],
       [0.36334926, 0.        , 0.        ]], dtype=float32)

In [146]:
c=np.zeros(a.T.shape)
at = a.T
for row in range(at.shape[0]):
    for col in range(at.shape[1]):
        c[row][col] = at[row][col]

In [147]:
at

array([[0.        , 0.56775475],
       [0.10336889, 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ]], dtype=float32)

In [148]:
c

array([[0.        , 0.56775475],
       [0.10336889, 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ]])

a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate3(__global  float* x,     // INPUT MATRIX DATA
                             __global  float* y,    // INPUT
                             uint topk,
                             uint msize,
                             __global  float* xout,    // INPUT
                             __global  uint* xoutidx,    // INPUT
                             __global  uint* youtidx    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint n = get_global_size(0);
      uint bs = get_global_size(1);
      uint gid2 = get_global_id(1);



      uint idx = n*gid2+gid;
      float valx = x[idx];
      uint posx = 0;
      for (uint i = 0; i < n; i++) {
        uint idx2 = n*gid2+i;
        float tempval = x[idx2];
        bool larger = tempval > valx;
        posx += (larger)?1:0;
      }
      
      uint idxy = n*gid2+gid;
      float valy = y[idx];
      uint posy = 0;
      for (uint i = 0; i < n; i++) {
        uint idx2 = n*gid2+i;
        float tempval2 = y[idx2];
        bool larger2 = tempval2 > valy;
        posy += (larger2)?1:0;
      }
      
      if (posx < topk) {
        xoutidx[posx+topk*gid2] = idx;
      }
      if (posy < topk) {
        youtidx[posy+topk*gid2] = idxy;
      }
      return;
      if (gid < topk) {
        for (uint j=0; j<topk; j++) {
          float res = x[xoutidx[gid+topk*gid2]+gid2*msize] * y[youtidx[j+topk*gid2]+gid2*msize];
          printf("\\nJ:%i  gid:(%i,%i)", j, gid, gid2);
          printf("\\nRES:%.2f - %i - %i -  %.2f - %.2f",res, xoutidx[gid+topk*gid2], youtidx[j+topk*gid2], x[xoutidx[gid+topk*gid2]+gid2*n], y[youtidx[j+topk*gid2]+gid2*n]);
          //barrier(CLK_GLOBAL_MEM_FENCE);
          xout[gid2*topk*topk+j*topk+gid] = res;
        }
      }
    }""").build()

In [243]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c.astype(np.float32))
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
x_sum_buf = cl.Buffer(ctx, mf.WRITE_ONLY, a.shape[0]*4)
y_sum_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.shape[1]*4)
x_cp_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*topk*4)
x_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)
y_idx_buf = cl.Buffer(ctx, mf.WRITE_ONLY, topk*4)

prg = cl.Program(ctx, """
    // sorts x and y in ascending order and returns sorted indices
    __kernel void genwupdate3(__global  float* x,     // INPUT MATRIX DATA
                              __global  float* y,    // INPUT
                              __global  float* xsum,    // INPUT
                              __global  float* ysum,    // INPUT
                              uint isize,
                              uint msize,
                              uint osize,
                              uint topk,
                              __global  float* xout,
                              __global  uint* xoutidx,   
                              __global  uint* youtidx    
                              ) { 
      uint gid = get_global_id(0);
      
      // get for a: sum axis0  b: sum axis1 then get topk
      ///////////////////////////////////////////////////
      if (gid < isize) {
        xsum[gid] = 0;
        for (uint i=0; i<isize; i++) {
          float val = x[gid*isize+i];
          if (gid == 0) {
            printf("\\nADD VALx: %.2f", val);
          }
          xsum[gid] += val;
        }
        
        float valx = xsum[gid];
        uint posx = 0;
        for (uint i = 0; i < isize; i++) {
          float tempval = xsum[i];
          bool larger = tempval > valx;
          posx += (larger)?1:0;
        }
        if (posx < topk) {
          xoutidx[posx] = gid;
        }
      }
      
      if (gid < osize) {
        ysum[gid] = 0;
        for (uint i=0; i<osize; i++) {
          float val = y[gid*osize+i];
          //if (gid == 3) {
          //  printf("\\nADD VALy: %.2f", val);
          //}
          ysum[gid] += val;
        }
        
        float valy = ysum[gid];
        uint posy = 0;
        for (uint i = 0; i < osize; i++) {
          float tempval = ysum[i];
          bool larger = tempval > valy;
          posy += (larger)?1:0;
        }
      
        if (posy < topk) {
          youtidx[posy] = gid;
        }
      }
      
      // only calc matrix multiplications for used grads
      ///////////////////////////////////////////////////
      if (gid < topk) {
        uint idxx = xoutidx[gid];
        for (uint j=0; j<topk; j++) {
          uint idxy = youtidx[j];
          xout[j*topk+gid] = 0;
          for (uint k=0; k<msize; k++) {
            uint xidx2 = isize*k+idxx;
            uint yidx2 = osize*k+idxy;
            xout[j*topk+gid] += x[xidx2] * y[yidx2];
            //if (gid == 0 && j == 1)
            //  printf("\\n ADD VAL:%.2f,%.2f - (%i,%i) - (%i,%i,%i)", x[xidx2], y[yidx2], idxx, idxy, gid, j, k);
          }
        }
      }
    }""").build()

In [244]:
a.shape, b.shape

((2, 4), (4, 3))

In [245]:
rows = a.shape[0]
msize = a.shape[1]

In [246]:
cols = b.shape[1]

In [247]:
mult = a.dot(b)

In [248]:
mult = mult.astype(np.float32)

In [249]:
res_buf = cl.Buffer(ctx, mf.READ_WRITE, np.prod([rows,b.shape[1]])*4)
knl = prg.genwupdate3  # Use this Kernel object for repeated calls
evt = knl(queue, [max(rows,cols)], None, a_buf, b_buf, x_sum_buf, y_sum_buf, np.uint32(rows), np.uint32(msize),np.uint32(cols), np.uint32(topk), x_cp_buf, x_idx_buf, y_idx_buf)

resxsum = np.zeros(a.shape[0]).astype(np.float32)
resysum = np.zeros(b.shape[1]).astype(np.float32)
resx = np.zeros(topk*topk).astype(np.float32)
resxidx = np.zeros(topk).astype(np.uint32)
resyidx = np.zeros(topk).astype(np.uint32)

cl.enqueue_copy(queue, resxsum, x_sum_buf)
cl.enqueue_copy(queue, resysum, y_sum_buf)
cl.enqueue_copy(queue, resx, x_cp_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf)
cl.enqueue_copy(queue, resyidx, y_idx_buf)


ADD VALx: 0.00
ADD VALx: 0.57

In [250]:
resx.reshape(topk,topk)

array([[0.        , 0.31604028],
       [0.0150735 , 0.        ]], dtype=float32)

In [251]:
a

array([[0.        , 0.10336889, 0.        , 0.        ],
       [0.56775475, 0.        , 0.        , 0.        ]], dtype=float32)

In [252]:
b.T

array([[0.        , 0.14582244, 0.        , 0.36334926],
       [0.        , 0.        , 0.        , 0.        ],
       [0.55664927, 0.        , 0.6065191 , 0.        ]], dtype=float32)

In [253]:
resxsum

array([0.56775475, 0.10336889], dtype=float32)

In [254]:
resysum

array([0.55664927, 0.14582244, 0.6065191 ], dtype=float32)

In [255]:
a.sum(axis=1)

array([0.10336889, 0.56775475], dtype=float32)

In [256]:
b.sum(axis=0)

array([0.5091717, 0.       , 1.1631684], dtype=float32)

In [257]:
mult

array([[0.0150735 , 0.        , 0.        ],
       [0.        , 0.        , 0.31604028]], dtype=float32)

In [258]:
resxidx

array([0, 1], dtype=uint32)

In [259]:
resyidx

array([2, 0], dtype=uint32)

In [260]:
idx = 1
xy0 = x[idx].reshape(dim,1)*y[idx]
xy0.shape

(8, 8)

In [261]:
xy0[0][0]

0.1063604

### update vals add dense

In [262]:
matadd = np.random.randn(*a.shape).astype(np.float32)
matadd

array([[ 0.87853116, -0.16395721, -0.6672961 ,  0.09980658],
       [ 0.32946754, -0.27029884,  1.7421029 ,  0.78900284]],
      dtype=float32)

In [263]:
a

array([[0.        , 0.10336889, 0.        , 0.        ],
       [0.56775475, 0.        , 0.        , 0.        ]], dtype=float32)

In [264]:
a_added = a + matadd

In [265]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
add_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=matadd)

prg = cl.Program(ctx, """
    // Every global_id_0 works on a row
    __kernel void adddense(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            float  lr,
                            uint   ellwidth,
                            uint   awidth,
                            __global  float* vector_x    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint nrows = get_global_size(0);

      uint nnz    = rowNnz[gid];
      uint baseidxs = gid*ellwidth;
      uint baseidxd = gid*awidth;
      
      for (uint i=0; i<awidth; i++) {
        float addval = vector_x[baseidxd+i];
        //if (gid==1)
        //  printf("\\nADD VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[baseidxs+i]);
        if (addval == 0) {
          continue;
        }
        if (i == colIdx[baseidxs+i]) {
          matData[baseidxs+i] += addval;
        } else {
          if (rowNnz[gid] >= ellwidth) {
            break;
          }
          if (i > colIdx[baseidxs+i]) {
            rowNnz[gid] += 1;
            //if (gid==1)
            //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
            matData[baseidxs+i] = addval;
            colIdx[baseidxs+i] = i;
            continue;
          }
          for (uint j=nnz; j>i; j--) {
            //printf("\\nMOVE:%.2f", matData[baseidx+j-1]);
            colIdx[baseidxs+j] = colIdx[baseidxs+j-1];
            matData[baseidxs+j] = matData[baseidxs+j-1];
          }
          rowNnz[gid] += 1;
          nnz = rowNnz[gid];
          //if (gid==1)
          //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
          matData[baseidxs+i] = addval;
          colIdx[baseidxs+i] = i;
          if (nnz >= ellwidth)
            break;
        }
      }
    }""").build()

In [266]:
a.shape, b.shape

((2, 4), (4, 3))

In [267]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [268]:
rows = a.shape[0]

In [269]:
mult = mult.astype(np.float32)

In [270]:
res_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)
knl = prg.adddense  # Use this Kernel object for repeated calls
knl(queue, [rows], None, adata_buf, acols_buf, annzs_buf, np.float32(1), np.uint32(ellwa),np.uint32(a.shape[1]), add_buf)

In [271]:
matadd[0][0]

0.87853116

In [272]:
data_res = np.empty_like(adata)
cols_res = np.empty_like(acols)
nnzs_res = np.empty_like(annz)
cl.enqueue_copy(queue, data_res, adata_buf)
cl.enqueue_copy(queue, cols_res, acols_buf)
cl.enqueue_copy(queue, nnzs_res, annzs_buf)

In [273]:
adenseadd = to_dense(data_res, cols_res, nnzs_res, ellwa, a.shape)
adenseadd

array([[0.        , 0.10336889, 0.        , 0.        ],
       [0.89722228, 0.        , 0.        , 0.        ]])

In [274]:
a

array([[0.        , 0.10336889, 0.        , 0.        ],
       [0.56775475, 0.        , 0.        , 0.        ]], dtype=float32)

In [275]:
matadd

array([[ 0.87853116, -0.16395721, -0.6672961 ,  0.09980658],
       [ 0.32946754, -0.27029884,  1.7421029 ,  0.78900284]],
      dtype=float32)

In [276]:
a_added

array([[ 0.87853116, -0.06058832, -0.6672961 ,  0.09980658],
       [ 0.8972223 , -0.27029884,  1.7421029 ,  0.78900284]],
      dtype=float32)

In [277]:
adenseadd == a_added

array([[False, False, False, False],
       [ True, False, False, False]])

### update vals

In [278]:
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)
add_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=matadd)

prg = cl.Program(ctx, """
    // Every global_id_0 works on a row
    __kernel void adddenset(__global  float* matData,     // INPUT MATRIX DATA
                            __global  uint*  colIdx,
                            __global  uint*  rowNnz,
                            float  lr,
                            uint   ellwidth,
                            uint   aheight,
                            __global  float* vector_x    // INPUT
                            ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint ncols = get_global_size(0);

      uint nnz    = rowNnz[gid];
      uint baseidxs = gid*ellwidth;
      
      for (uint i=0; i<aheight; i++) {
        if (nnz > ellwidth)
            break;
        uint baseidxd = i*ncols+gid;
        float addval = vector_x[baseidxd];
        //if (gid==1)
        //  printf("\\nADD VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[baseidxs+i]);
        if (addval == 0) {
          continue;
        }
        if (i == colIdx[baseidxs+i]) {
          printf("\\nADD VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
          matData[baseidxs+i] += addval;
        } else {
          if (rowNnz[gid] >= ellwidth) {
            break;
          }
          if (i > colIdx[baseidxs+i]) {
            rowNnz[gid] += 1;
            //if (gid==1)
            //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
            matData[baseidxs+i] = addval;
            colIdx[baseidxs+i] = i;
            continue;
          }
          for (uint j=nnz; j>i; j--) {
            //printf("\\nMOVE:%.2f", matData[baseidx+j-1]);
            colIdx[baseidxs+j] = colIdx[baseidxs+j-1];
            matData[baseidxs+j] = matData[baseidxs+j-1];
          }
          rowNnz[gid] += 1;
          nnz = rowNnz[gid];
          //if (gid==1)
          //  printf("\\nSET VAL:%.2f idx:%i/%i  col:%i", addval, baseidxs+i, baseidxd+i, colIdx[i]);
          matData[baseidxs+i] = addval;
          colIdx[baseidxs+i] = i;
        }
      }
    }""").build()

In [279]:
a.shape, b.shape

((2, 4), (4, 3))

In [280]:
res = np.zeros(a.shape[0]).astype(np.float32)
#res

In [281]:
cols = a.shape[1]

In [282]:
mult = mult.astype(np.float32)

In [283]:
res_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)
knl = prg.adddenset  # Use this Kernel object for repeated calls
knl(queue, [cols], None, adatat_buf, acolst_buf, annzst_buf, np.float32(1), np.uint32(ellwat),np.uint32(a.T.shape[1]), add_buf)


ADD VAL:-0.16 idx:1/1  col:1
ADD VAL:-0.67 idx:2/2  col:1
ADD VAL:0.10 idx:3/3  col:1

In [284]:
matadd[0][0]

0.87853116

In [285]:
datat_res = np.empty_like(adatat)
colst_res = np.empty_like(acolst)
nnzst_res = np.empty_like(annzt)
cl.enqueue_copy(queue, datat_res, adatat_buf)
cl.enqueue_copy(queue, colst_res, acolst_buf)
cl.enqueue_copy(queue, nnzst_res, annzst_buf)

In [286]:
adenseaddt = to_dense(datat_res, colst_res, nnzst_res, ellwat, a.T.shape).T
adenseaddt

array([[ 0.        , -0.06058832, -0.66729611,  0.        ],
       [ 0.56775475,  0.        ,  0.        ,  1.74210286]])

In [287]:
a

array([[0.        , 0.10336889, 0.        , 0.        ],
       [0.56775475, 0.        , 0.        , 0.        ]], dtype=float32)

In [288]:
matadd

array([[ 0.87853116, -0.16395721, -0.6672961 ,  0.09980658],
       [ 0.32946754, -0.27029884,  1.7421029 ,  0.78900284]],
      dtype=float32)

In [289]:
a_added

array([[ 0.87853116, -0.06058832, -0.6672961 ,  0.09980658],
       [ 0.8972223 , -0.27029884,  1.7421029 ,  0.78900284]],
      dtype=float32)

In [290]:
adenseaddt == a_added

array([[False,  True,  True, False],
       [False, False, False, False]])

### Make Random

In [291]:
rand = SparseTensor.uniform(2,4)
rand

<SparseTensor <GPUBuffer with shape (8,)> with grad None>

In [292]:
rand.to_numpy()

array([[0.00897554, 0.0054974 , 0.00514649, 0.00844749],
       [0.001605  , 0.00695174, 0.00135912, 0.00781948]])

In [293]:
rand.data

<GPUBuffer with shape (8,)>

### update vals

In [302]:
adata_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adata)
acols_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acols)
annzs_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annz)

In [303]:
prg = cl.Program(ctx, """
// Every global_id_0 works on a row
    __kernel void addvals(__global  float* matData,     // INPUT MATRIX DATA
                         __global  uint*  colIdx,
                         __global  uint*  rowNnz,
                         float lr,
                         uint   ellwidth,
                         __global  float* updatevals,    // INPUT
                         __global  uint* updatexidx,
                         __global  uint* updateyidx
                         ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint gid2 = get_global_id(1);
      uint topk = get_global_size(0);
      uint bs = get_global_size(1);
      uint baseupdateidx = topk*topk*gid2;
      uint baseidxidx = topk*gid2;
      uint col = updateyidx[baseidxidx+gid];

      for (uint i=0; i<topk; i++) {
        float val = updatevals[baseupdateidx+gid*topk+i];
        uint row = updatexidx[baseidxidx+i];
        for (uint i=0; i<rowNnz[row]; i++) {
          uint idx = row*ellwidth+i;
          if (colIdx[idx] >= col) {
            //printf("\\nFOUND:%i/%i  - idx:%i", colIdx[idx], col, idx);
            if (colIdx[idx] == col) {
              matData[idx] += -val*lr;
              printf("\\nUPDATE[%i,%i]: %f", row,col, val);
              break;
            } else {
              // insert new column
              printf("\\nINSERT[%i,%i]: %.2f", row,col, val);
              for (uint j=rowNnz[row]+1; j>i; j--) {
                uint idx2 = row*ellwidth+j;
                matData[idx2] = matData[idx2-1];
                colIdx[idx2] = colIdx[idx2-1];
              }
              matData[idx] = -val*lr;
              colIdx[idx] = col;
              rowNnz[row] += 1;
              break;
            }
          }
        }
        if (rowNnz[row] >= ellwidth) {
          break;
        }
      }
    }""").build()

In [304]:
knl = prg.addvals  # Use this Kernel object for repeated calls
knl(queue, [topk,1], None, adata_buf, acols_buf, annzs_buf, np.float32(1), np.uint32(ellwa), x_cp_buf, x_idx_buf, y_idx_buf)

resa = np.empty_like(adata)
resaidx = np.zeros(acols.shape).astype(np.uint32)
resannz = np.zeros(annz.shape).astype(np.uint32)

cl.enqueue_copy(queue, resa, adata_buf)
cl.enqueue_copy(queue, resaidx, acols_buf)
cl.enqueue_copy(queue, resannz, annzs_buf)


INSERT[0,0]: 0.02

In [305]:
resa.shape, resaidx.shape, resannz.shape, ellwa, a.T.shape

((2,), (2,), (2,), 1, (4, 2))

In [309]:
adenseadd = to_dense(resa, resaidx, resannz, ellwa, a.shape)
adenseadd

array([[-0.0150735 ,  0.10336889,  0.        ,  0.        ],
       [ 0.        ,  0.10336889,  0.        ,  0.        ]])

In [310]:
adenseadd - adense

array([[-0.0150735 ,  0.        ,  0.        ,  0.        ],
       [-0.56775475,  0.10336889,  0.        ,  0.        ]])

In [311]:
adenseadd == adense

array([[False,  True,  True,  True],
       [False, False,  True,  True]])

In [312]:
ellwa

1

In [313]:
adata2 = adata.reshape(-1, ellwa)
adata2

array([[0.10336889],
       [0.56775475]], dtype=float32)

In [314]:
resa = resa.reshape(-1, ellwa)
resa

array([[-0.0150735 ],
       [ 0.10336889]], dtype=float32)

In [315]:
resa - adata2

array([[-0.11844239],
       [-0.46438587]], dtype=float32)

In [316]:
acols

array([1, 0], dtype=uint32)

In [317]:
resaidx

array([0, 1], dtype=uint32)

In [318]:
resannz

array([2, 1], dtype=uint32)

In [319]:
annz

array([1, 1], dtype=uint32)

### update vals2

In [ ]:
adatat_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=adatat)
acolst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=acolst)
annzst_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=annzt)

In [ ]:
prg = cl.Program(ctx, """
// Every global_id_0 works on a row
    __kernel void addvals(__global  float* matData,     // INPUT MATRIX DATA
                         __global  uint*  colIdx,
                         __global  uint*  rowNnz,
                         float lr,
                         uint   ellwidth,
                         __global  float* updatevals,    // INPUT
                         __global  uint* updatexidx,
                         __global  uint* updateyidx
                         ) { // LOCAL SHARED BUFFER
      uint gid = get_global_id(0);
      uint gid2 = get_global_id(1);
      uint topk = get_global_size(0);
      uint bs = get_global_size(1);
      uint baseupdateidx = topk*topk*gid2;
      uint baseidxidx = topk*gid2;
      uint row = updateyidx[baseidxidx+gid];

      for (uint i=0; i<topk; i++) {
        float val = updatevals[baseupdateidx+gid*topk+i];
        uint col = updatexidx[baseidxidx+i];
        for (uint i=0; i<rowNnz[row]; i++) {
          uint idx = row*ellwidth+i;
          if (colIdx[idx] >= col) {
            //printf("\\nFOUND:%i/%i  - idx:%i", colIdx[idx], col, idx);
            if (colIdx[idx] == col) {
              matData[idx] += -val*lr;
              printf("\\nUPDATE[%i,%i]: %f", row,col, val);
              break;
            } else {
              // insert new column
              printf("\\nINSERT[%i,%i]: %.2f", row,col, val);
              for (uint j=rowNnz[row]+1; j>i; j--) {
                uint idx2 = row*ellwidth+j;
                matData[idx2] = matData[idx2-1];
                colIdx[idx2] = colIdx[idx2-1];
              }
              matData[idx] = -val*lr;
              colIdx[idx] = col;
              rowNnz[row] += 1;
              break;
            }
          }
        }
        if (rowNnz[row] >= ellwidth) {
          break;
        }
      }
    }""").build()

In [ ]:
knl = prg.addvals  # Use this Kernel object for repeated calls
knl(queue, [topk,bs], None, adatat_buf, acolst_buf, annzst_buf, np.float32(1), np.uint32(ellwat), x_cp_buf, x_idx_buf, y_idx_buf)

resat = np.empty_like(adatat)
resaidxt = np.zeros(acolst.shape).astype(np.uint32)
resannzt = np.zeros(annzt.shape).astype(np.uint32)

cl.enqueue_copy(queue, resat, adatat_buf)
cl.enqueue_copy(queue, resaidxt, acolst_buf)
cl.enqueue_copy(queue, resannzt, annzst_buf)

In [ ]:
ellwa

In [ ]:
resat.shape, resaidxt.shape, resannzt.shape

In [ ]:
adenseaddt = to_dense(resat, resaidxt, resannzt, ellwat, a.T.shape)
adenseaddt

In [ ]:
adenseadd == adenseaddt.T

In [ ]:
adata2t = adatat.reshape(-1, ellwat)
adata2t

In [ ]:
resat = resat.reshape(-1, ellwat)
resat

In [ ]:
resat - adata2t

In [ ]:
acols

In [ ]:
resaidx

In [ ]:
resannz

In [ ]:
annz

# OTHER

import numpy as np
import pyopencl as cl

mf = cl.mem_flags

dim = 16
topk = 4

x = np.random.rand(dim).astype(np.float32)
y = np.random.rand(dim).astype(np.float32)
x.shape,y.shape

dim1 = 4
dim2 = 8
dim3 = 1

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx,
        properties=cl.command_queue_properties.PROFILING_ENABLE)

sparsity = 0.2

a = np.zeros((dim1,dim2))
b = np.random.rand(dim2,dim3).flatten().astype(np.float32)

a.shape, b.shape

In [ ]:
x_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x)
y_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y)
val_out_buf = cl.Buffer(ctx, mf.READ_WRITE, 4*topk*topk)
x_idx_buf = cl.Buffer(ctx, mf.READ_WRITE, topk*4)
y_idx_buf = cl.Buffer(ctx, mf.READ_WRITE, topk*4)

prg = cl.Program(ctx, """
// Every global_id_0 works on a row
__kernel void genwupdate2(__global  float* x,     // INPUT MATRIX DATA
                         __global  float* y,    // INPUT
                         __global  float* xout,    // INPUT
                         uint topk,
                         __global  uint* xoutidx,    // INPUT
                         __global  uint* youtidx    // INPUT
                        ) { // LOCAL SHARED BUFFER
  uint gid = get_global_id(0);
  uint n = get_global_size(0);
  
  xout[gid] = x[gid];
  xoutidx[gid] = gid;
  youtidx[gid] = gid;
  
  float valx = x[gid];
  float valy = y[gid];
  uint posx = 0;
  uint posy = 0;
  for (uint i = 0; i < n; i++) {
    float tempval = x[i];
    float tempval2 = y[i];
    bool larger = tempval > valx;
    bool larger2 = tempval2 > valy;
      
    posx += (larger)?1:0;
    posy += (larger2)?1:0;
  }
  //printf("posx:%i", posx);
  if (posx < topk) {
    xoutidx[posx] = gid;
  }
  if (posy < topk) {
    youtidx[posy] = gid;
  }
  if (gid < topk) {
    uint i = gid;
    for (uint j=0; j<topk; j++) {
      xout[gid*topk+j] = x[xoutidx[gid]] * y[youtidx[j]];
    }
  }
}""").build()

In [ ]:
knl = prg.genwupdate2  # Use this Kernel object for repeated calls
event = knl(queue, [dim,], None, x_buf, y_buf, val_out_buf, np.uint32(topk), x_idx_buf, y_idx_buf)

#event.wait()
val_out = np.zeros(topk*topk).astype(np.float32)
resxidx = np.zeros(topk).astype(np.uint32)
resyidx = np.zeros(topk).astype(np.uint32)

cl.enqueue_copy(queue, val_out, val_out_buf)
cl.enqueue_copy(queue, resxidx, x_idx_buf, wait_for=[event])
cl.enqueue_copy(queue, resyidx, y_idx_buf)

In [ ]:
val_out

In [ ]:
resxidx

In [ ]:
resyidx

In [ ]:
asdf

In [ ]:
from __future__ import division

KERNEL_CODE = """
// Thread block size
#define BLOCK_SIZE %(block_size)d
// Matrix dimensions
// (chosen as multiples of the thread block size for simplicity)
#define WA %(w_a)d // Matrix A width
#define HA %(h_a)d // Matrix A height
#define WB %(w_b)d // Matrix B width
#define HB WA  // Matrix B height
#define WC WB  // Matrix C width
#define HC HA  // Matrix C height
/*
 * Copyright 1993-2009 NVIDIA Corporation.  All rights reserved.
 *
 * NVIDIA Corporation and its licensors retain all intellectual property and
 * proprietary rights in and to this software and related documentation.
 * Any use, reproduction, disclosure, or distribution of this software
 * and related documentation without an express license agreement from
 * NVIDIA Corporation is strictly prohibited.
 *
 * Please refer to the applicable NVIDIA end user license agreement (EULA)
 * associated with this source code for terms and conditions that govern
 * your use of this NVIDIA software.
 *
 */
/* Matrix multiplication: C = A * B.
 * Device code.
 */
#define AS(j, i) As[i + j * BLOCK_SIZE]
#define BS(j, i) Bs[i + j * BLOCK_SIZE]
////////////////////////////////////////////////////////////////////////////////
//! Matrix multiplication on the device: C = A * B
//! WA is A's width and WB is B's width
////////////////////////////////////////////////////////////////////////////////
__kernel __attribute__((reqd_work_group_size(16,16,1))) 
void
matrixMul( __global float* C, __global float* A, __global float* B)
{
    __local float As[BLOCK_SIZE*BLOCK_SIZE];
    __local float Bs[BLOCK_SIZE*BLOCK_SIZE];
    // Block index
    int bx = get_group_id(0);
    int by = get_group_id(1);
    // Thread index
    int tx = get_local_id(0);
    int ty = get_local_id(1);
    // Index of the first sub-matrix of A processed by the block
    int aBegin = WA * BLOCK_SIZE * by;
    // Index of the last sub-matrix of A processed by the block
    int aEnd   = aBegin + WA - 1;
    // Step size used to iterate through the sub-matrices of A
    int aStep  = BLOCK_SIZE;
    // Index of the first sub-matrix of B processed by the block
    int bBegin = BLOCK_SIZE * bx;
    // Step size used to iterate through the sub-matrices of B
    int bStep  = BLOCK_SIZE * WB;
    // Csub is used to store the element of the block sub-matrix
    // that is computed by the thread
    float Csub = 0.0f;
    // Loop over all the sub-matrices of A and B
    // required to compute the block sub-matrix
    for (int a = aBegin, b = bBegin;
             a <= aEnd;
             a += aStep, b += bStep) {
        // Load the matrices from device memory
        // to shared memory; each thread loads
        // one element of each matrix
        AS(ty, tx) = A[a + WA * ty + tx];
        BS(ty, tx) = B[b + WB * ty + tx];
        // Synchronize to make sure the matrices are loaded
        barrier(CLK_LOCAL_MEM_FENCE);
        // Multiply the two matrices together;
        // each thread computes one element
        // of the block sub-matrix
        for (int k = 0; k < BLOCK_SIZE; ++k)
            Csub += AS(ty, k) * BS(k, tx);
        // Synchronize to make sure that the preceding
        // computation is done before loading two new
        // sub-matrices of A and B in the next iteration
        barrier(CLK_LOCAL_MEM_FENCE);
    }
    // Write the block sub-matrix to device memory;
    // each thread writes one element
    C[get_global_id(1) * get_global_size(0) + get_global_id(0)] = Csub;
}
"""


In [ ]:
a2 = np.random.rand(4,4)

In [ ]:
a2

In [ ]:
a2.sum(axis=1)

In [ ]:
b2 = np.random.rand(4,4)

In [ ]:
b2

In [ ]:
b2.sum(axis=0)

In [ ]:
matmul = a2.dot(b2)
matmul